In [1]:
from tensorflow.keras.layers import AveragePooling2D,Flatten,Dropout,Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

In [2]:
le_rate=1e-4
bs=32
epochs=20

## Train dataset

In [3]:
dir1="seg_train/seg_train/"
categories=os.listdir(dir1)

In [4]:
train_data=[]
train_labels=[]

for category in categories:
    path=os.path.join(dir1,category)

    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image=load_img(img_path,target_size=(150,150))
        image=img_to_array(image)
        image=preprocess_input(image)

        train_data.append(image)
        train_labels.append(category)

In [15]:
c=0
for i in train_labels:
    if i=="forest":
        c+=1

c

2271

In [16]:
train_labels[c-1]

'forest'

In [17]:
encoder=LabelEncoder()
train_labels=encoder.fit_transform(train_labels)
train_labels=to_categorical(train_labels)

train_data=np.array(train_data,dtype="float32")
train_labels=np.array(train_labels)


In [ ]:
train_labels[2190]

## Test Dataset

In [17]:
dir2="../input/intel-image-classification/seg_test/seg_test/"

In [18]:
test_data=[]
test_labels=[]

for category in categories:
    path=os.path.join(dir2,category)

    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image=load_img(img_path,target_size=(150,150))
        image=img_to_array(image)
        image=preprocess_input(image)

        test_data.append(image)
        test_labels.append(category)

In [20]:
test_labels=encoder.transform(test_labels)
test_labels=to_categorical(test_labels)

test_data=np.array(test_data,dtype="float32")
test_labels=np.array(test_labels)

In [21]:
image_gen=ImageDataGenerator(rotation_range=20,shear_range=0.2,horizontal_flip=True)

In [39]:
basemodel=MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(150,150,3)))

In [40]:
headmodel=basemodel.output
headmodel=AveragePooling2D(pool_size=(2,2))(headmodel)
headmodel=Flatten(name="flatten")(headmodel)
headmodel=Dense(units=120,activation="relu")(headmodel)
headmodel=Dropout(0.5)(headmodel)
headmodel=Dense(units=len(categories),activation="softmax")(headmodel)

In [41]:
model=Model(inputs=basemodel.input,outputs=headmodel)

for layers in basemodel.layers:
    basemodel.trainable=False

In [42]:
opt=Adam(lr=le_rate,decay=le_rate/epochs)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
model.fit(image_gen.flow(train_data,train_labels,batch_size=bs),
            steps_per_epoch=len(train_data)//bs,
            validation_data=(test_data,test_labels),
            validation_steps=len(test_data)//bs,
            epochs=epochs)

In [43]:
model.save("natural_image_model.h5")

In [2]:
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

In [4]:
categories=["Buildings","Forest","Glacier","Mountain","Sea","Street"]

model=load_model("natural_image_model.h5")
image=load_img("seg_test/seg_test/forest/20384.jpg",target_size=(150,150))
image=img_to_array(image)
image=preprocess_input(image)
image=np.expand_dims(image,axis=0)
image=np.array(image,dtype="float32")
categories[np.argmax(model.predict(image))]

'Forest'